## 블로그 리뷰 크롤링

In [1]:
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
import datetime
from datetime import timedelta
import json
import time
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotVisibleException
import pandas as pd 

 ### **지점번호, 방문자 더보기xpath, 블로그 더보기 xpath --> 엑셀에 저장해놓기**

In [56]:
store_number_lst = ['21726749', '34582591','1670762252']
store_name_lst = ['투썸플레이스 종로구청점','투썸플레이스 종각역점','투썸플레이스 종로알파빌딩점']
see_more_visit_xpath_lst = ['//*[@id="app-root"]/div/div/div[2]/div[5]/div/div[4]/div[2]/a','//*[@id="app-root"]/div/div/div[2]/div[5]/div/div[8]/div[2]/a','//*[@id="app-root"]/div/div/div[2]/div[5]/div/div[4]/div[2]/a']
see_more_blog_xpath_lst = ['//*[@id="app-root"]/div/div/div[2]/div[5]/div/div[9]/div[2]/a']

In [52]:
store_number = store_number_lst[1] 
options = webdriver.ChromeOptions()
# options.add_argument('headless')#창 안 뜨게 설정
review_driver = webdriver.Chrome('./chromedriver.exe',options=options)# 크롬 경로
review_driver.implicitly_wait(3)

#url 접속
review_url = 'https://pcmap.place.naver.com/restaurant/'+store_number+'/home?from=map&amp;fromPanelNum=2&amp'
review_driver.get(review_url)
time.sleep(5)



In [50]:
#홈에서의 더보기 경로는 홈페이지마다 위치가 달라지므로 f12 버튼 눌러서 파악해야 함
def review_seen(review_driver, see_more_xpath):
    #홈에서 스크롤 다운
    review_driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    #홈에서 방문자 리뷰 더보기 클릭
    review_driver.find_element_by_xpath(see_more_xpath).click()

def all_visit_review_seen(review_driver):
    #방문자 리뷰에서 더보기 탭 끝까지 클릭하기 
    while True:
        try:
            review_driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div[2]/div[5]/div[4]/div[5]/div[2]/a').click()
        except:
            break

def five_blog_review_seen(review_driver):
    # 블로그 더보기 탭 누르기 5번
    for _ in range(5):
        try:
            review_driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div[2]/div[5]/div[4]/div[2]/div[2]/a').click()
        except:
            break

In [9]:
# 3년 이내의 블로그 리뷰만 링크 받기
def blog_link_list(blog_soup):
    global html_cleaner
    blog_url_lst=[]
    date_lst= []
    before_3y = datetime.datetime.now() - timedelta(days = 365*3)
    for s in blog_soup.find_all('li',{'class':'_3OyGD'}):
        date = s.find('span',{'class':'_7OvgA'})
        date = re.sub(html_cleaner,"", str(date))
        #3년 이내 블로그 글만 퍼오기
        if pd.to_datetime(date) > before_3y:
            url = s.find('a')['href']
            blog_url_lst.append(url)
            date_lst.append(date)
    blog_df = pd.DataFrame({'url':blog_url_lst, 'date':date_lst})
    blog_df['content'] = ' '
    return blog_df


In [37]:
def blog_review_crawling(blog_df):
    options = webdriver.ChromeOptions()
    options.add_argument('headless')#창 안 뜨게 설정

    blog_driver = webdriver.Chrome('./chromedriver.exe',options=options)# 크롬 경로
    blog_driver.implicitly_wait(5)
    
    for url in blog_df['url']:
        blog_driver.get(url)
        time.sleep(5)
        blog_driver.switch_to.frame('mainFrame')
        content = ''
        overlays = ".se-component.se-text.se-l-default" # 내용 크롤링
        unrefined_content = blog_driver.find_elements_by_css_selector(overlays)
        
        if len(unrefined_content) == 0:
            overlays = ".se_component.se_paragraph.default" # 내용 크롤링
            unrefined_content = blog_driver.find_elements_by_css_selector(overlays)
        for c in unrefined_content:
            content = content + c.text # content 라는 값에 + 하면서 점점 누적
            content = content.replace("\n","")
        blog_df.loc[blog_df['url']==url,'content'] = content

    return blog_df

In [54]:
# 블로그 리뷰 탭으로 넘어가기
review_seen(review_driver, see_more_blog_xpath_lst[0])
# 5번 더보기 탭 누르기
five_blog_review_seen(review_driver)
# 현재 html 받기
blog_html = review_driver.page_source
blog_soup = BeautifulSoup(blog_html,'html.parser')
html_cleaner = re.compile('<.*?>')

#블로그 링크, 날짜 --> 블로그 df에 저장
blog_df = blog_link_list(blog_soup)
#블로그 내용 크롤링 --> 블로그 df에 저장
blog_df = blog_review_crawling(blog_df)
blog_df.insert(0,'지점명',store_name_lst[0],allow_duplicates=False)

In [58]:
blog_df

,지점명,url,date,content
0,투썸플레이스 종로구청점,https://blog.naver.com/jungkeung/221785728643,2020.01.26.,집🏡여유를 즐겼다오랜만에 카레 만들어 먹었당 ㅎㅎ엄마가 갑자기 취업을 하게 돼서청소...
1,투썸플레이스 종로구청점,https://blog.naver.com/jungkeung/221811707729,2020.02.17.,오늘은 쏙캠 회계폭탄 처리를 위한 회의..약간 마음의 준비를 하고 갔다 ㅋㅋㅋ잠 못...
2,투썸플레이스 종로구청점,https://blog.naver.com/gptjd1212/222503637351,2021.09.16.,가을의 대표 작물인 고구마!투썸에서 9월을 맞이하여 고구마 디저트가 나왔더라고요 ㅎ...
3,투썸플레이스 종로구청점,https://blog.naver.com/tlrevolution/221391828275,2018.11.05.,갑자기 내린 비를 피해서 방문하게 된 투썸플레이스 종각역점.. 요즘 투썸에 소홀했는...
4,투썸플레이스 종로구청점,https://blog.naver.com/all10x10/222517371861,2021.09.26.,밀크바 팥빙수가 없어서베리바 요거 케이크 빙수랑 인절미 티라미슈 먹었다ㅡ빙수에 산딸...
5,투썸플레이스 종로구청점,https://blog.naver.com/orro2/222403948699,2021.06.20.,투썸플레이스아임얼라이브 콤부차요즘 콤부차 많이 드시죠?저는 콤부차의 첫 인식경로가기...
